In [ ]:
pwd() # cd("G:\\My Drive\\_MOPJ\\Julia\\ROM4SM\\")

In [ ]:
readdir()

# Proper Orthogonal Decomposition (POD) for Structural Mechanics
Use Case : Frame made up of plane elastic truss (11 rods et 5 unconstrained nodes with 2 degrees of freedom (10 dof)). The displacement is fixed at node 6 and node 7 fixée. The load is applied on nodes 1&2 with respectivelely a force of -5000N and a force of -2000N (see figure below)  

# Frame

<img src="https://raw.githubusercontent.com/amdeld/ROM4SM.jl/master/img/TrussStructure_LBC.png" alt="Drawing" style="width: 700px;"/>


## Stiffness Matrix Function for a plane elastic truss element 

In [1]:
function trstiff(coo,E,A)
# Function that computes stiffness matrix for truss element
# (file name trstiff.m)
L=sqrt((coo[2,1]-coo[1,1])^2+(coo[2,2]-coo[1,2])^2)
C=(coo[2,1]-coo[1,1])/L
S=(coo[2,2]-coo[1,2])/L
# Stiffness matrix
STF=E*A/L*[C^2 S*C -C^2 -S*C;
S*C S^2 -S*C -S^2;
-C^2 -S*C C^2 S*C;
-S*C -S^2 S*C S^2];
return(STF)
end

trstiff (generic function with 1 method)

## Assembly Function

In [2]:
# Main program for truss structure
# Coordinates - units L[mm]
function truss_str(E,A)
COO=[0 0; 2 0; 4 0 ;1 1; 3 1; -1 1; 5 1]
COO=1000*COO
DOF=size(COO,1)*2
# Elements
ELM=[1 2; 2 3; 6 4; 4 5; 5 7; 1 6; 1 4; 2 4; 2 5; 3 5; 3 7]
# Properties (units F[N], L[mm])
#E=125000
#A=6
# Assembling of stiffness matrix
MSTF=zeros(DOF,DOF)
X=zeros(2,2)
for i in 1:size(ELM,1)
X[1,:]=COO[ELM[i,1],:]
X[2,:]=COO[ELM[i,2],:]
STF=trstiff(X,E,A)
MSTF[2*ELM[i,1]-1:2*ELM[i,1],2*ELM[i,1]-
1:2*ELM[i,1]]=MSTF[2*ELM[i,1]-1:2*ELM[i,1],2*ELM[i,1]-
1:2*ELM[i,1]]+STF[1:2,1:2]
MSTF[2*ELM[i,1]-1:2*ELM[i,1],2*ELM[i,2]-
1:2*ELM[i,2]]=MSTF[2*ELM[i,1]-1:2*ELM[i,1],2*ELM[i,2]-
1:2*ELM[i,2]]+STF[3:4,1:2]
MSTF[2*ELM[i,2]-1:2*ELM[i,2],2*ELM[i,1]-
1:2*ELM[i,1]]=MSTF[2*ELM[i,2]-1:2*ELM[i,2],2*ELM[i,1]-
1:2*ELM[i,1]]+STF[1:2,3:4]
MSTF[2*ELM[i,2]-1:2*ELM[i,2],2*ELM[i,2]-
1:2*ELM[i,2]]=MSTF[2*ELM[i,2]-1:2*ELM[i,2],2*ELM[i,2]-
1:2*ELM[i,2]]+STF[3:4,3:4]
end
# Constrains
con=[12 13 14 15] # constrained DOF
CDOF=size(con,2)
# Forces
dF=zeros(DOF,1)
dF[2]=-5000
dF[6]=-2000
# Reduced stiffness matrix & force vector
RSTF=MSTF[1:DOF-CDOF,1:DOF-CDOF]
dFR=dF[1:DOF-CDOF]
# Solving for displacements
d=inv(RSTF)*dFR
# End of main program
return(d)
end

truss_str (generic function with 1 method)

## Design Of Experiments function (doe)

In [3]:
function doe()
    cnt = 1
    U=zeros(10)
    for E in [125000 150000 175000 200000]
        for A in [6 8 10 12]
            Disp = truss_str(E, A)
            #show(Disp)
            if cnt > 1
                U = [U Disp]
            else
                U = Disp
            end
            cnt += 1
        end
    end
    #show(U)
    return (U)
end

doe (generic function with 1 method)

## DOE execution for the snaphot matrix creation
<img src="https://raw.githubusercontent.com/amdeld/ROM4SM.jl/master/img/snaphotmatrix4FEM.png" alt="Drawing" style="width: 700px;"/>

$Y=\left[\begin{array}{cccccccccc}
| & | & | & | & | & | & | & | & | & |\\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots\\
| & | & | & | & | & | & | & | & | & |\\
y_{1} & y_{2} & y_{3} & y_{4} & y_{5} & y_{6} & y_{7} & y_{8} & y_{9} & y_{10}\\
| & | & | & | & | & | & | & | & | & |\\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots\\
| & | & | & | & | & | & | & | & | & |
\end{array}\right]$

Range of Young modulus E = {125000; 150000; 175000; 200000} (unit MPa)
Range of cross section A = {6; 8; 10; 12} (unit mm)

In [4]:
# Execute The design of experiments (DOE)
Y=doe()
println("\rThe Data - Snaphot matrix Y coming out from the DOE\r")
show(stdout,"text/plain", round.(Y;digits=2))

The Data - Snaphot matrix Y coming out from the DOE
10×16 Array{Float64,2}:
 -11.56   -8.67   -6.93   -5.78   -9.63   -7.22   -5.78   -4.81   -8.25   -6.19   -4.95   -4.13   -7.22   -5.42   -4.33   -3.61
 -28.53  -21.39  -17.12  -14.26  -23.77  -17.83  -14.26  -11.89  -20.38  -15.28  -12.23  -10.19  -17.83  -13.37  -10.7    -8.91
  -0.89   -0.67   -0.53   -0.44   -0.74   -0.56   -0.44   -0.37   -0.63   -0.48   -0.38   -0.32   -0.56   -0.42   -0.33   -0.28
 -31.87  -23.9   -19.12  -15.93  -26.55  -19.92  -15.93  -13.28  -22.76  -17.07  -13.66  -11.38  -19.92  -14.94  -11.95   -9.96
   7.11    5.33    4.27    3.56    5.93    4.44    3.56    2.96    5.08    3.81    3.05    2.54    4.44    3.33    2.67    2.22
 -16.54  -12.4    -9.92   -8.27  -13.78  -10.34   -8.27   -6.89  -11.81   -8.86   -7.09   -5.91  -10.34   -7.75   -6.2    -5.17
  -2.67   -2.0    -1.6    -1.33   -2.22   -1.67   -1.33   -1.11   -1.9    -1.43   -1.14   -0.95   -1.67   -1.25   -1.0    -0.83
 -35.53  -26.65  -21.32  -17

## Verification of the results for the first column (E=125000 and A=6mm) with the commercial Finite element software MSC.Marc
<img src="https://raw.githubusercontent.com/amdeld/ROM4SM.jl/master/img/TrussStructure_DisplacementX.png" alt="Drawing" style="width: 700px;"/>
<img src="https://raw.githubusercontent.com/amdeld/ROM4SM.jl/master/img/TrussStructure_DisplacementY.png" alt="Drawing" style="width: 700px;"/>

## POD 2 methodes are used here which should give the same results (Eigen decomposition and Singular Value Decompostion) followed by reconstruction and reduction steps

In [5]:
using LinearAlgebra
# Chargement du paquet LinearAlgebra pour les opérations d'algèbre linéaire

In [6]:
F=eigen(Y*Y')
println("\rEigenValues vector λ\r")
F.values

EigenValues vector λ


10-element Array{Float64,1}:
    -2.2135126648768165e-12
    -1.1048887716527176e-12
    -4.3125039855839144e-13
    -1.10895170442201e-13
    -1.816827865912878e-14
    -9.806735284196114e-15
     1.5177084586265563e-15
     1.853963908321237e-12
     2.1827872842550278e-11
 24580.216532550152

In [7]:
println("\rEigenVectors matrix ϕ\r")
ϕ=F.vectors
show(stdout,"text/plain", round.(ϕ;digits=2))

EigenVectors matrix ϕ
10×10 Array{Float64,2}:
  0.18   0.12   0.07  -0.78  -0.29  -0.47   0.06   0.05  -0.08  -0.18
 -0.28   0.26   0.32  -0.04  -0.03   0.22  -0.03  -0.68  -0.2   -0.43
  0.01   0.0   -0.02  -0.06   0.07  -0.05  -0.99   0.02  -0.01  -0.01
  0.57  -0.07  -0.54   0.24  -0.05  -0.05   0.0   -0.17  -0.23  -0.48
  0.24   0.19  -0.19  -0.46   0.08   0.79   0.0    0.09   0.05   0.11
 -0.36  -0.82  -0.15  -0.27   0.1    0.13   0.02   0.01  -0.12  -0.25
  0.09   0.11   0.02  -0.16   0.94  -0.23   0.09  -0.06  -0.02  -0.04
 -0.19   0.25   0.18   0.12   0.05   0.12   0.01   0.7   -0.25  -0.54
  0.58  -0.37   0.71   0.07   0.02   0.13  -0.02   0.03  -0.02  -0.04
  0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.9   -0.43

In [8]:
println("\rAmplitude matrix A\r")
A=F.vectors'*Y
show(stdout,"text/plain", round.(A;digits=2))

Amplitude matrix A
10×16 Array{Float64,2}:
  0.0   -0.0   -0.0    0.0   -0.0   -0.0    0.0   -0.0   -0.0   -0.0    0.0  -0.0   -0.0   -0.0   -0.0   -0.0
 -0.0    0.0   -0.0   -0.0   -0.0    0.0   -0.0   -0.0   -0.0   -0.0   -0.0  -0.0    0.0   -0.0    0.0    0.0
  0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   -0.0   0.0    0.0    0.0    0.0    0.0
 -0.0   -0.0    0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0   -0.0    0.0  -0.0   -0.0   -0.0   -0.0   -0.0
 -0.0    0.0   -0.0   -0.0    0.0    0.0   -0.0    0.0    0.0    0.0   -0.0   0.0    0.0    0.0    0.0    0.0
 -0.0    0.0   -0.0   -0.0   -0.0    0.0   -0.0   -0.0   -0.0   -0.0   -0.0  -0.0    0.0   -0.0    0.0    0.0
 -0.0   -0.0    0.0   -0.0   -0.0    0.0   -0.0   -0.0   -0.0   -0.0    0.0  -0.0    0.0   -0.0   -0.0    0.0
  0.0    0.0    0.0    0.0   -0.0    0.0    0.0   -0.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0  

In [9]:
println("\rReconstruction of Yrecons=ϕA\r")
Yrecons=ϕ*A
show(stdout,"text/plain", round.(Yrecons;digits=2))

Reconstruction of Yrecons=ϕA
10×16 Array{Float64,2}:
 -11.56   -8.67   -6.93   -5.78   -9.63   -7.22   -5.78   -4.81   -8.25   -6.19   -4.95   -4.13   -7.22   -5.42   -4.33   -3.61
 -28.53  -21.39  -17.12  -14.26  -23.77  -17.83  -14.26  -11.89  -20.38  -15.28  -12.23  -10.19  -17.83  -13.37  -10.7    -8.91
  -0.89   -0.67   -0.53   -0.44   -0.74   -0.56   -0.44   -0.37   -0.63   -0.48   -0.38   -0.32   -0.56   -0.42   -0.33   -0.28
 -31.87  -23.9   -19.12  -15.93  -26.55  -19.92  -15.93  -13.28  -22.76  -17.07  -13.66  -11.38  -19.92  -14.94  -11.95   -9.96
   7.11    5.33    4.27    3.56    5.93    4.44    3.56    2.96    5.08    3.81    3.05    2.54    4.44    3.33    2.67    2.22
 -16.54  -12.4    -9.92   -8.27  -13.78  -10.34   -8.27   -6.89  -11.81   -8.86   -7.09   -5.91  -10.34   -7.75   -6.2    -5.17
  -2.67   -2.0    -1.6    -1.33   -2.22   -1.67   -1.33   -1.11   -1.9    -1.43   -1.14   -0.95   -1.67   -1.25   -1.0    -0.83
 -35.53  -26.65  -21.32  -17.76  -29.61  -22.21  -1

In [10]:
println("\rReduction of Yapprox=ϕ10*A10'≈ Y\r")
Yapprox=ϕ[:,10]*A[10,:]'
show(stdout,"text/plain", round.(Yapprox;digits=2))

Reduction of Yapprox=ϕ10*A10'≈ Y
10×16 Array{Float64,2}:
 -11.56   -8.67   -6.93   -5.78   -9.63   -7.22   -5.78   -4.81   -8.25   -6.19   -4.95   -4.13   -7.22   -5.42   -4.33   -3.61
 -28.53  -21.39  -17.12  -14.26  -23.77  -17.83  -14.26  -11.89  -20.38  -15.28  -12.23  -10.19  -17.83  -13.37  -10.7    -8.91
  -0.89   -0.67   -0.53   -0.44   -0.74   -0.56   -0.44   -0.37   -0.63   -0.48   -0.38   -0.32   -0.56   -0.42   -0.33   -0.28
 -31.87  -23.9   -19.12  -15.93  -26.55  -19.92  -15.93  -13.28  -22.76  -17.07  -13.66  -11.38  -19.92  -14.94  -11.95   -9.96
   7.11    5.33    4.27    3.56    5.93    4.44    3.56    2.96    5.08    3.81    3.05    2.54    4.44    3.33    2.67    2.22
 -16.54  -12.4    -9.92   -8.27  -13.78  -10.34   -8.27   -6.89  -11.81   -8.86   -7.09   -5.91  -10.34   -7.75   -6.2    -5.17
  -2.67   -2.0    -1.6    -1.33   -2.22   -1.67   -1.33   -1.11   -1.9    -1.43   -1.14   -0.95   -1.67   -1.25   -1.0    -0.83
 -35.53  -26.65  -21.32  -17.76  -29.61  -22.21

In [11]:
y1=ϕ[:,10]*A[10,:][1]
println("\ry1=\r")
show(stdout,"text/plain", round.(y1;digits=2))

y1=
10-element Array{Float64,1}:
 -11.56
 -28.53
  -0.89
 -31.87
   7.11
 -16.54
  -2.67
 -35.53
  -2.67
 -28.2

In [17]:
y2=ϕ[:,10]*A[10,:][2]
println("\ry2=\r")
show(stdout,"text/plain", round.(y1;digits=2))

y2=
10-element Array{Float64,1}:
  -8.67
 -21.39
  -0.67
 -23.9
   5.33
 -12.4
  -2.0
 -26.65
  -2.0
 -21.15

In [12]:
#Calculate the SVD of the snapshot matrix
Z=svd(Y)
println("\rSVD decomposition of the snaphot matrix=\r")
Z

SVD decomposition of the snaphot matrix=


SVD{Float64,Float64,Array{Float64,2}}
U factor:
10×10 Array{Float64,2}:
  0.175013   -0.0769962    0.0634878  …  -0.117766    -0.534239    0.179672
  0.432037   -0.452066     0.705132      -0.095187     0.051515   -0.0430513
  0.0134625   0.127815     0.180737      -0.515135     0.505987   -0.145575
  0.482621    0.254724    -0.248985      -0.103211    -0.120392   -0.0891002
 -0.1077     -0.204958     0.064601      -0.00713723  -0.477135    0.246816
  0.250492    0.253263     0.131573   …  -0.262671    -0.254016   -0.151175
  0.0403875  -0.125426     0.187767       0.66208      0.243976    0.113673
  0.538104   -0.471607    -0.564768       0.0115301    0.198933    0.0388636
  0.0403875  -0.00734608   0.0161755      0.29713     -0.215104   -0.881018
  0.427138    0.603438     0.167731       0.324404     0.0621886   0.249673
singular values:
10-element Array{Float64,1}:
 156.78079133793835
   2.275930897213532e-14
   1.878342523872848e-14
   9.451247149500198e-15
   6.9373281104171084e-1

In [13]:
println("\rCheck the reconstruction of Yrecons=UΣV* \nYrecons=")
Yrecons=Z.U*diagm(Z.S)*Z.Vt
show(stdout,"text/plain", round.(Yrecons;digits=2))

Check the reconstruction of Yrecons=UΣV* 
Yrecons=
10×16 Array{Float64,2}:
 -11.56   -8.67   -6.93   -5.78   -9.63   -7.22   -5.78   -4.81   -8.25   -6.19   -4.95   -4.13   -7.22   -5.42   -4.33   -3.61
 -28.53  -21.39  -17.12  -14.26  -23.77  -17.83  -14.26  -11.89  -20.38  -15.28  -12.23  -10.19  -17.83  -13.37  -10.7    -8.91
  -0.89   -0.67   -0.53   -0.44   -0.74   -0.56   -0.44   -0.37   -0.63   -0.48   -0.38   -0.32   -0.56   -0.42   -0.33   -0.28
 -31.87  -23.9   -19.12  -15.93  -26.55  -19.92  -15.93  -13.28  -22.76  -17.07  -13.66  -11.38  -19.92  -14.94  -11.95   -9.96
   7.11    5.33    4.27    3.56    5.93    4.44    3.56    2.96    5.08    3.81    3.05    2.54    4.44    3.33    2.67    2.22
 -16.54  -12.4    -9.92   -8.27  -13.78  -10.34   -8.27   -6.89  -11.81   -8.86   -7.09   -5.91  -10.34   -7.75   -6.2    -5.17
  -2.67   -2.0    -1.6    -1.33   -2.22   -1.67   -1.33   -1.11   -1.9    -1.43   -1.14   -0.95   -1.67   -1.25   -1.0    -0.83
 -35.53  -26.65  -21.32  -17.

In [14]:
println("\rThe approximation Ỹ is based on the first energetic mode")
println("\rỸ=")
Ỹ=Z.U[:,1]*(diagm(Z.S)*Z.Vt)[1,:]'
show(stdout,"text/plain", round.(Ỹ;digits=2))

The approximation Ỹ is based on the first energetic mode
Ỹ=
10×16 Array{Float64,2}:
 -11.56   -8.67   -6.93   -5.78   -9.63   -7.22   -5.78   -4.81   -8.25   -6.19   -4.95   -4.13   -7.22   -5.42   -4.33   -3.61
 -28.53  -21.39  -17.12  -14.26  -23.77  -17.83  -14.26  -11.89  -20.38  -15.28  -12.23  -10.19  -17.83  -13.37  -10.7    -8.91
  -0.89   -0.67   -0.53   -0.44   -0.74   -0.56   -0.44   -0.37   -0.63   -0.48   -0.38   -0.32   -0.56   -0.42   -0.33   -0.28
 -31.87  -23.9   -19.12  -15.93  -26.55  -19.92  -15.93  -13.28  -22.76  -17.07  -13.66  -11.38  -19.92  -14.94  -11.95   -9.96
   7.11    5.33    4.27    3.56    5.93    4.44    3.56    2.96    5.08    3.81    3.05    2.54    4.44    3.33    2.67    2.22
 -16.54  -12.4    -9.92   -8.27  -13.78  -10.34   -8.27   -6.89  -11.81   -8.86   -7.09   -5.91  -10.34   -7.75   -6.2    -5.17
  -2.67   -2.0    -1.6    -1.33   -2.22   -1.67   -1.33   -1.11   -1.9    -1.43   -1.14   -0.95   -1.67   -1.25   -1.0    -0.83
 -35.53  -26.65  -

In [15]:
y1=Z.U[:,1]*(Z.S[1]*Z.Vt[1,:][1])
println("\ry1=\r")
show(stdout,"text/plain", round.(y1;digits=2))

y1=
10-element Array{Float64,1}:
 -11.56
 -28.53
  -0.89
 -31.87
   7.11
 -16.54
  -2.67
 -35.53
  -2.67
 -28.2

In [18]:
y2=Z.U[:,1]*(Z.S[1]*Z.Vt[1,:][2])
println("\ry2=\r")
show(stdout,"text/plain", round.(y2;digits=2))

y2=
10-element Array{Float64,1}:
  -8.67
 -21.39
  -0.67
 -23.9
   5.33
 -12.4
  -2.0
 -26.65
  -2.0
 -21.15

For arbitray parameters, the combined use of radial basis functions (RBF) is required